<a href="https://colab.research.google.com/github/pmishra77276/Card-Classification/blob/main/NER98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!kaggle datasets download -d debasisdotcom/name-entity-recognition-ner-dataset
import zipfile
zip_ref = zipfile.ZipFile('/content/name-entity-recognition-ner-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

cp: cannot stat 'kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/debasisdotcom/name-entity-recognition-ner-dataset
License(s): other
  0% 0.00/3.17M [00:00<?, ?B/s]
100% 3.17M/3.17M [00:00<00:00, 106MB/s]


we have to calculate at word level to get good predictions

In [12]:
import pandas as pd
df=pd.read_csv('/content/NER dataset.csv' ,encoding= 'unicode_escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [13]:
number_of_words=len(df['Word'].unique())
number_of_words

35178

In [14]:
number_of_tags=len(df['Tag'].unique())
number_of_tags

17

In [16]:
df.ffill(axis=0,inplace=True)

In [19]:
df.isnull().sum()
words = list(set(df["Word"].values))
words.append("ENDPAD")
num_words = len(words)

tags = list(set(df["Tag"].values))
num_tags = len(tags)

import plotly.express as px

fig = px.histogram(df[~df.Tag.str.contains("O")], x="Tag",color="Tag")
fig.show()

In [21]:
import tqdm
def sentence_integrate(data):
  agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
  return data.groupby('Sentence #').apply(agg_func).tolist()

sentences=sentence_integrate(df)

sentences[0]

<ipython-input-21-56283cab6157>:6: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [23]:
word_idx={}
tag_idx={}
for i,j in enumerate(words):
  word_idx[j]=i+1
for i,j in enumerate(tags):
  tag_idx[j]=i

In [36]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 50

X = []
for i in sentences:
  li=[]
  for k in i:
    li.append(word_idx[k[0]])
  X.append(li)
len(X)
X=pad_sequences(X,maxlen=max_len,padding='post')
y = [[tag_idx[w[2]] for w in s] for s in sentences]
y=pad_sequences(y,maxlen=max_len,padding='post')

In [34]:
len(df['Sentence #'].unique())

47959

In [38]:
X[0].shape

(50,)

# **Model Building**

In [37]:
import tensorflow
from tensorflow.keras.layers import Bidirectional,LSTM,Dense,Embedding
from tensorflow.keras.models import Sequential


In [48]:
model=Sequential([
    Embedding(num_words,50,input_shape=X[0].shape),
    Bidirectional(LSTM(50,return_sequences=True)),
    Dense(50,activation='softmax')
])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 50, 50)              │       1,758,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 50, 100)             │          40,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50, 50)              │           5,050 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,804,350 (6.88 MB)

 Trainable params: 1,804,350 (6.88 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
hist=model.fit(X,y,epochs=3,validation_split=0.2)

Epoch 1/3
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 102s 82ms/step - accuracy: 0.9068 - loss: 0.4881 - val_accuracy: 0.9801 - val_loss: 0.0696
Epoch 2/3
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 137s 78ms/step - accuracy: 0.9840 - loss: 0.0561 - val_accuracy: 0.9838 - val_loss: 0.0538
Epoch 3/3
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 95s 79ms/step - accuracy: 0.9890 - loss: 0.0370 - val_accuracy: 0.9846 - val_loss: 0.0505


In [76]:
# print("Evaluate on test data")
# results = model.evaluate(x_test, y_test, batch_size=128)
# print("test loss: {} ".format(results[0]))
# print("test accuracy: {} ".format(results[1]))
import numpy as np
i = np.random.randint(0, X.shape[0])
print("This is sentence:",i)
print(X[i])
p = model.predict(np.array([X[i]]))
print(p)
p = np.argmax(p, axis=-1)
print(p)

print("{:15}{:5}\t {}\n".format("Word", "True", "Pred"))
print("-" *30)
for w, true, pred in zip(X[i], y[i], p[0]):
    print("{:15}{}\t{}".format(words[w-1], tags[true], tags[pred]))

This is sentence: 38723
[ 2187 21633 10023 34932 18665 15502 12810 11104 10769 24200 17281 34850
 28245  1639 34055 14924 10258 31637 27782 32940 24800  4350 32321 10258
 10023 18247 14169 34122 24108 10329 12810    60 15814  9472  1829 31884
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[[[1.8236769e-04 1.6471596e-02 1.6071246e-05 ... 1.0560875e-06
   9.0706510e-07 1.0576517e-06]
  [4.7900194e-05 2.8171808e-01 1.9912775e-05 ... 1.1649593e-06
   8.9656260e-07 1.2899744e-06]
  [6.0844563e-05 7.7924466e-05 2.1473867e-05 ... 1.7199420e-07
   3.1699292e-07 2.6711177e-07]
  ...
  [9.9998766e-01 1.4121462e-08 4.9302406e-07 ... 1.7180797e-08
   3.2836875e-09 7.7022158e-09]
  [9.9998140e-01 1.9139154e-08 8.3605914e-07 ... 3.3360660e-08
   6.7109442e-09 1.5652866e-08]
  [9.9984396e-01 1.4865348e-07 7.9872361e-06 ... 3.5788420e-07
   1.0287964e-07 2.4993710e-07]]]
[[15  4 15 15 15 15 15 15 15 11 15  8 13  6 13 15 15 1

In [83]:
# from tensorflow.keras.utils import pad_sequence
txt=input()

txt1=[]
for i in txt.split():
  try:
    txt1.append(word_idx[i])
  except:
    txt1.append(0)

txt1=pad_sequences([txt1],maxlen=max_len,padding='post')
print(txt1[0])

y_pred=model.predict(np.array(txt1))

y_pred=np.argmax(y_pred,axis=-1)
print(y_pred)
for i in y_pred[0]:

  for k,l in tag_idx.items():
    if l==i:
      print(k)
      break

Amazon is also called as Lungs of Earth situated in Brazil
[  110 33756 22214 12536  1007     0 29163 23642     0 33352 14380     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[[ 1 15 15 15 15  0 15  4  0 15  4  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]]
B-org
O
O
O
O
B-eve
O
B-geo
B-eve
O
B-geo
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve
B-eve


In [68]:
tag_idx

{'B-eve': 0,
 'B-org': 1,
 'I-nat': 2,
 'I-eve': 3,
 'B-geo': 4,
 'B-art': 5,
 'I-per': 6,
 'B-nat': 7,
 'B-gpe': 8,
 'I-gpe': 9,
 'I-org': 10,
 'B-tim': 11,
 'I-tim': 12,
 'B-per': 13,
 'I-geo': 14,
 'O': 15,
 'I-art': 16}